In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!git clone https://github.com/resemble-ai/Resemblyzer.git

Cloning into 'Resemblyzer'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 595 (delta 2), reused 1 (delta 0), pack-reused 586
Receiving objects: 100% (595/595), 101.46 MiB | 21.70 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
!pip install webrtcvad-wheels

     |████████████████████████████████| 92kB 5.0MB/s 


In [ ]:
cd Resemblyzer

/content/Resemblyzer


In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path




#give the file path to your audio file
audio_file_path = '/content/drive/My Drive/Dual Rnn/speaker1.wav'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
#print(type(wav))



encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.04 seconds.
(782, 256)


In [ ]:
!pip3 install spectralcluster

In [ ]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)

In [ ]:
def create_labelling(labels,wav_splits):
   # from resemblyzer import sampling_rate
    sampling_rate=16000
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [ ]:
labelling

[('0', 0, 15.74),
 ('1', 15.74, 19.16),
 ('0', 19.16, 29.54),
 ('1', 29.54, 29.72),
 ('0', 29.72, 33.92),
 ('1', 33.92, 36.86),
 ('0', 36.86, 46.94),
 ('1', 46.94, 47.48),
 ('0', 47.48, 47.66)]

In [ ]:
type(labelling)

list

In [1]:
#Creating different audios based on the labelling/timeframes received.
output = {}
for x, y, z in labelling:
    if x in output:
        output[x].append((y, z))
    else:
        output[x] = [(y, z)]

#keys = []
values = []
items = output.items()
for item in items:
    values.append(item[1])

print(values[0]) 
#print(values[1])   


NameError: ignored

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_wav('/content/drive/My Drive/Dual Rnn/speaker1.wav')
voices = []

voices = []
n = 0
for i in values:
    for j in i:
        start_time, stop_time = j
        n += audio[start_time:stop_time]
    voices.append(n)

for i, j in enumerate(voices):
    j.export(f'/content/drive/My Drive/Dual Rnn/voice_{i}.wav')


In [ ]:
import numpy as np
from scipy.io.wavfile import write

# data = np.random.uniform(-1,1,44100) # 44100 random samples between -1 and 1
# scaled = np.int16(wav/np.max(np.abs(wav)) * 32767)
write('test.wav', 8000, wav)

In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = '/content/drive/My Drive/Colab Notebooks/1.wav'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.02 seconds.
(367, 256)


In [ ]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=3,
    max_clusters=100,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)

In [ ]:
def create_labelling(labels,wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [ ]:
labelling

[('2', 0, 3.44),
 ('1', 3.44, 9.74),
 ('0', 9.74, 12.8),
 ('1', 12.8, 12.98),
 ('0', 12.98, 13.1),
 ('2', 13.1, 13.58),
 ('0', 13.58, 16.28),
 ('2', 16.28, 18.98),
 ('0', 18.98, 22.76)]